In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex2 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


## Code for Connecion and Query

In [3]:
def get_dataset_ref(client, project_bq, dataset_bq):    
    dataset_ref = client.dataset(dataset_bq, project=project_bq)    
    dataset = client.get_dataset(dataset_ref)
    return dataset_ref, dataset

def get_table_ref(client, dataset_ref, table_name):    
    table_ref = dataset_ref.table(table_name)    
    table = client.get_table(table_ref)
    return table_ref, table

def execute_query(client, query):    
    # Set up the query (cancel the query if it would use too much of 
    # your quota, with the limit set to 10 GB)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
    query_job = client.query(query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    return query_job.to_dataframe()
    

In [4]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data") #REFERENCE
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)                              #FECTH

# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")  #REFERENCE
# API request - fetch the table
table = client.get_table(table_ref)                  #FETCH

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  app.launch_new_instance()


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


In [5]:
print(dataset.description)
print('\nTables:')
print([table.table_id for table in list(client.list_tables(dataset))])
print('\nSchema:\n',table.schema)

OpenAQ is an open-source project to surface live, real-time air quality data from around the world. Their “mission is to enable previously impossible science, impact policy and empower the public to fight air pollution.” The data includes air quality measurements from 5490 locations in 47 countries.

Scientists, researchers, developers, and citizens can use this data to understand the quality of air near them currently. The dataset only includes the most current measurement available for the location (no historical data). 

Dataset Source: openaq.org

Category: Science

Use: This dataset is publicly available for anyone to use under the following terms provided by the Dataset Source — https://openaq.org/#/about?_k=s3aspo — and is provided "AS IS" without any warranty, express or implied, from Google. Google disclaims all liability for any damages, direct or indirect, resulting from the use of the dataset. 

Update Frequency: Hourly

Tables:
['global_air_quality']

Schema:
 [SchemaField

# 1. Select Excercise

### 1.1 Which countries have reported pollution levels in units of "ppm"? In the code cell below, set first_query to an SQL query that pulls the appropriate entries from the country column.

In case it's useful to see an example query, here's some code from the tutorial:
```
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """
```       

In [6]:
query_1 = """ Select country
from `bigquery-public-data.openaq.global_air_quality`
where unit="ppm" """

In [8]:
# Query to select countries with units of "ppm"
query = query_1
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
query_results = query_job.to_dataframe()

# View top few rows of results
print(query_results.head())

  country
0      AR
1      IL
2      IL
3      AR
4      AR


### 2.1 Which pollution levels were reported to be exactly 0?

* Set zero_pollution_query to select all columns of the rows where the value column is 0.
* Set zero_pollution_results to a pandas DataFrame containing the query results.

In [9]:
query_2 = """ Select *
from `bigquery-public-data.openaq.global_air_quality`
where value=0 """

In [10]:
# Query to select countries with units of "ppm"
query = query_2
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
zero_pollution_results = query_job.to_dataframe()

# View top few rows of results
print(zero_pollution_results.head())
# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


                     location          city country pollutant  value  \
0  Żary, ul. Szymanowskiego 8          Żary      PL        bc    0.0   
1     Starachowice, ul. Złota  Starachowice      PL        bc    0.0   
2       Kraków, ul. Bulwarowa        Kraków      PL        bc    0.0   
3   Zielonka, Bory Tucholskie      Zielonka      PL        bc    0.0   
4   Żagań, ul. Kochanowskiego         Żagań      PL        bc    0.0   

                  timestamp   unit source_name  latitude  longitude  \
0 2022-05-05 02:00:00+00:00  µg/m³        GIOS       1.0  51.642656   
1 2022-05-08 11:00:00+00:00  µg/m³        GIOS       1.0  51.050611   
2 2022-05-07 13:00:00+00:00  µg/m³        GIOS       1.0  50.069308   
3 2022-05-15 11:00:00+00:00  µg/m³        GIOS       1.0  53.662136   
4 2022-05-02 13:00:00+00:00  µg/m³        GIOS       1.0  51.615447   

   averaged_over_in_hours       location_geom  
0               15.127808  POINT(51.642656 1)  
1               21.084175  POINT(51.050611 1

<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> You still need to define the following variables: `zero_pollution_query`

# 2. Groupby Excercise

In [11]:
# Construct a reference to the "hacker_news" dataset
dataset_hack_ref = client.dataset("hacker_news", project="bigquery-public-data")
# API request - fetch the dataset
dataset_hack = client.get_dataset(dataset_hack_ref)

# Construct a reference to the "comments" table
table_com_ref = dataset_hack_ref.table("comments")
# API request - fetch the table
table_com = client.get_table(table_com_ref)

# Preview the first five lines of the "comments" table
df_head_table_com = client.list_rows(table_com, max_results=20).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  if sys.path[0] == "":


In [12]:
df_head_table_com

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0
5,3867418,Iv,Iv,1334921984,2012-04-20 11:39:44+00:00,&#62; There's a whole class of 'mom &#38; pop'...,3867404,None,None,0
6,3925617,Iv,Iv,1336076765,2012-05-03 20:26:05+00:00,I'm also in this ballpark (300-600€ / day) whi...,3924840,None,None,0
7,3107534,Iv,Iv,1318520044,2011-10-13 15:34:04+00:00,how do you run unity in non-accelerated mode ?...,3107241,None,None,0
8,8409259,Iv,Iv,1412421647,2014-10-04 11:20:47+00:00,Polio is not exterminated even if it is absent...,8409226,None,None,0
9,2855741,Jd,Jd,1312690646,2011-08-07 04:17:26+00:00,"Yep, I didn't find Rescuetime very helpful. I ...",2855343,None,None,0


### 2.1 Prolific commenters
Hacker News would like to send awards to everyone who has written more than 10,000 posts. Write a query that returns all authors with more than 10,000 posts as well as their post counts. Call the column with post counts NumPosts.



In [15]:
query_1 = """ Select author, count(id) as NumPosts
from `bigquery-public-data.hacker_news.comments`
group by author 
having NumPosts > 10000"""
execute_query(client, query_1)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,author,NumPosts
0,eru,10448
1,rbanffy,10557
2,DanBC,12902
3,sp332,10882
4,davidw,10764
5,rayiner,11080
6,tptacek,33839
7,jacquesm,21107
8,jrockway,13626
9,anigbrowl,11395


We use **Having** to filter information after the groupby is done. We also can filter results before the groupby using **where** 

In [16]:
query_2 = """ Select author, count(id) as NumPosts
from `bigquery-public-data.hacker_news.comments`
where CHAR_LENGTH(text) > 100
group by author 
having NumPosts > 10000"""

df_result_2 = execute_query(client, query_2)
df_result_2

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,author,NumPosts
0,DanBC,11460
1,tptacek,27188
2,jacquesm,16980
3,jrockway,10526
4,dragonwriter,10467


The remainded question is, Is posible to use **where** rahter than **having** to filter information after the **group by** command?

In [17]:
query_3 = """ Select * from 
(Select author, count(id) as NumPosts
from `bigquery-public-data.hacker_news.comments`
where CHAR_LENGTH(text) > 100
group by author) as table_tmp 
where NumPosts > 10000"""

df_result_3 = execute_query(client, query_3)
df_result_3

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,author,NumPosts
0,dragonwriter,10467
1,DanBC,11460
2,tptacek,27188
3,jacquesm,16980
4,jrockway,10526


### 2.2 Deleted comments
How many comments have been deleted? (If a comment was deleted, the deleted column in the comments table will have the value True.)

In [18]:
query_4 = """ Select deleted, count(id) as NumDeleted
from `bigquery-public-data.hacker_news.comments`
where deleted = True
group by deleted
"""
df_result_4 = execute_query(client, query_4)
df_result_4

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,deleted,NumDeleted
0,True,227736


In [19]:
query_5 = """ Select count(1) as NumDeleted
from `bigquery-public-data.hacker_news.comments`
where deleted = TRUE
group by deleted
"""

df_result_5 = execute_query(client, query_5)
df_result_5

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,NumDeleted
0,227736


# 3. Order By

In [20]:
# Create a "Client" object
client = bigquery.Client()
dataset_ref, dataset = get_dataset_ref(client, "bigquery-public-data", "world_bank_intl_education")
table_ref, table = get_table_ref(client, dataset_ref, "international_education")
df_head_table = client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """


In [21]:
df_head_table

,country_name,country_code,indicator_name,indicator_code,value,year
0,Cabo Verde,CPV,Population of the official age for lower secon...,SP.SEC.LTOT.IN,2.977300e+04,2016
1,Canada,CAN,Population of the official age for upper secon...,SP.SEC.UTOT.IN,1.220331e+06,2016
2,Chad,TCD,"Labor force, total",SL.TLF.TOTL.IN,5.411585e+06,2016
3,Djibouti,DJI,"Enrolment in secondary general, female (number)",SE.SEC.ENRL.GC.FE,2.504000e+04,2016
4,Sub-Saharan Africa (excluding high income),SSA,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,7.361327e+00,2016


### 3.1 Government expenditure on education
The value in the indicator_code column describes what type of data is shown in a given row.
One interesting indicator code is SE.XPD.TOTL.GD.ZS, which corresponds to "Government expenditure on education as % of GDP (%)".

Which countries spend the largest fraction of GDP on education?

To answer this question, consider only the rows in the dataset corresponding to indicator code SE.XPD.TOTL.GD.ZS, and write a query that returns the average value in the value column for each country in the dataset between the years 2010-2017 (including 2010 and 2017 in the average).

Requirements:

Your results should have the country name rather than the country code. You will have one row for each country.
The aggregate function for average is AVG(). Use the name avg_ed_spending_pct for the column created by this aggregation.
Order the results so the countries that spend the largest fraction of GDP on education show up first.
In case it's useful to see a sample query, here's a query you saw in the tutorial (using a different dataset):

#### Query to find out the number of accidents for each day of the week
```
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """
```

In [22]:
query_6 = """
        SELECT country_code, country_name, avg(value) as avg_ed_spending_pct             
        FROM `bigquery-public-data.world_bank_intl_education.international_education` 
        where indicator_code = 'SE.XPD.TOTL.GD.ZS' and year>=2010 and year<=2017
        group by country_code, country_name
        order by avg_ed_spending_pct desc
        """
        
df_result_6 = execute_query(client, query_6)
df_result_6    
#     where indicator_code = 'SE.XPD.TOTL.GD.ZS' and year between 2010 and 2017

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,country_code,country_name,avg_ed_spending_pct
0,CUB,Cuba,12.837270
1,FSM,"Micronesia, Fed. Sts.",12.467750
2,SLB,Solomon Islands,10.001080
3,MDA,Moldova,8.372153
4,NAM,Namibia,8.349610
...,...,...,...
152,KHM,Cambodia,1.706404
153,PSE,West Bank and Gaza,1.503760
154,SSD,South Sudan,1.409726
155,MCO,Monaco,1.409606


In [23]:
query_6 = """
        SELECT country_code, country_name, avg(value) as avg_ed_spending_pct            
        FROM `bigquery-public-data.world_bank_intl_education.international_education` 
        where indicator_code = 'SE.XPD.TOTL.GD.ZS' and year between 2010 and 2017
        group by country_code, country_name
        order by avg_ed_spending_pct desc
        """
        
df_result_6 = execute_query(client, query_6)
df_result_6    

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,country_code,country_name,avg_ed_spending_pct
0,CUB,Cuba,12.837270
1,FSM,"Micronesia, Fed. Sts.",12.467750
2,SLB,Solomon Islands,10.001080
3,MDA,Moldova,8.372153
4,NAM,Namibia,8.349610
...,...,...,...
152,KHM,Cambodia,1.706404
153,PSE,West Bank and Gaza,1.503760
154,SSD,South Sudan,1.409726
155,MCO,Monaco,1.409606


### 3.2 Identify interesting codes to explore
The last question started by telling you to focus on rows with the code SE.XPD.TOTL.GD.ZS. But how would you find more interesting indicator codes to explore?

There are 1000s of codes in the dataset, so it would be time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, you might restrict yourself to codes that are reported by many countries.

Write a query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.

Requirements:

* You should have one row for each indicator code.
* The columns in your results should be called indicator_code, indicator_name, and num_rows.
* Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
* To get both the indicator_code and indicator_name in your resulting DataFrame, you need to include both in your SELECT statement (in addition to a COUNT() aggregation). This requires you to include both in your GROUP BY clause.
* Order from results most frequent to least frequent.

In [24]:
query_7 = """
        select * from 
        (SELECT indicator_code, indicator_name, count(1) as num_rows            
        FROM `bigquery-public-data.world_bank_intl_education.international_education` 
        where year = 2016
        group by indicator_code, indicator_name) as table_tmp
        where num_rows >= 175
        order by num_rows desc
        """
        
df_result_7 = execute_query(client, query_7)
df_result_7    

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,indicator_code,indicator_name,num_rows
0,SP.POP.TOTL,"Population, total",232
1,SP.POP.GROW,Population growth (annual %),232
2,IT.NET.USER.P2,Internet users (per 100 people),223
3,SP.POP.1564.MA.IN,"Population, ages 15-64, male",213
4,SP.POP.0014.FE.IN,"Population, ages 0-14, female",213
5,SP.POP.0014.TO.ZS,"Population, ages 0-14 (% of total)",213
6,SP.POP.TOTL.FE.ZS,"Population, female (% of total)",213
7,SP.POP.1564.TO,"Population, ages 15-64, total",213
8,SH.DYN.MORT,"Mortality rate, under-5 (per 1,000)",213
9,SP.POP.1564.FE.IN,"Population, ages 15-64, female",213


#### To avoid to use the second **where** creating and an additional table as the above, we can use **havig** as follow:

In [25]:
query_7 = """        
        SELECT indicator_code, indicator_name, count(1) as num_rows            
        FROM `bigquery-public-data.world_bank_intl_education.international_education` 
        where year = 2016
        group by indicator_code, indicator_name
        having num_rows >= 175
        order by num_rows desc
        """
        
df_result_7 = execute_query(client, query_7)
df_result_7    

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,indicator_code,indicator_name,num_rows
0,SP.POP.TOTL,"Population, total",232
1,SP.POP.GROW,Population growth (annual %),232
2,IT.NET.USER.P2,Internet users (per 100 people),223
3,SP.POP.1564.MA.IN,"Population, ages 15-64, male",213
4,SP.POP.0014.FE.IN,"Population, ages 0-14, female",213
5,SP.POP.0014.TO.ZS,"Population, ages 0-14 (% of total)",213
6,SP.POP.TOTL.FE.ZS,"Population, female (% of total)",213
7,SP.POP.1564.TO,"Population, ages 15-64, total",213
8,SH.DYN.MORT,"Mortality rate, under-5 (per 1,000)",213
9,SP.POP.1564.FE.IN,"Population, ages 15-64, female",213


# 4. As and With

In [41]:
# Create a "Client" object
client = bigquery.Client()
dataset_ref, dataset = get_dataset_ref(client, "bigquery-public-data", "chicago_taxi_trips")
# table_ref, table = get_table_ref(client, dataset_ref, "taxi_trips")
# df_head_table = client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


In [28]:
# show table in the dataset
for table in list(client.list_tables(dataset)):
    print(table.table_id)

taxi_trips


In [42]:
table_ref, table = get_table_ref(client, dataset_ref, "taxi_trips")
df_head_table = client.list_rows(table, max_results=5).to_dataframe()
df_head_table

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,7b3e74418111fef19fd73835853d9c1cebd727d5,fa62820eb95bd0b9cc1a0aaf6cf26730aee22ec578db91...,2014-04-01 16:00:00+00:00,2014-04-01 16:00:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,6.45,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
1,9d2eefe122d32a385207087a71dd33deed0eea96,89612cda57630834fb854b4e42206d3478145557d975ce...,2014-03-06 21:30:00+00:00,2014-03-06 21:30:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,12.05,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
2,a8410244d022d258ae357f6ede25d0162efd675f,36dee7e88e2048dfeb8a98c9e420a1600544a8d4f21fe6...,2014-04-18 10:45:00+00:00,2014-04-18 10:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,12.55,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
3,c590e1ea0a0998541e049de913e4a26499955123,85d86e614d844575634820c6e19498d1cfd59bdecc396b...,2015-02-05 15:30:00+00:00,2015-02-05 15:45:00+00:00,1020,2.5,NaN,NaN,NaN,NaN,...,0.0,9.85,Cash,None,NaN,NaN,None,NaN,NaN,None
4,bdd767ee32e7038e4c1bcac28cd516cc7cb549b8,f4bc2acfbb68d505b7ca37f8553ab1fc28a2686429b55d...,2014-03-28 09:45:00+00:00,2014-03-28 09:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,16.45,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None


### 4.1 Determine when this data is from
If the data is sufficiently old, we might be careful before assuming the data is still relevant to traffic patterns today. Write a query that counts the number of trips in each year.

Your results should have two columns:

* year - the year of the trips
* num_trips - the number of trips in that year

Hints:

* When using GROUP BY and ORDER BY, you should refer to the columns by the alias year that you set at the top of the SELECT query.
* The SQL code to SELECT the year from trip_start_timestamp is SELECT EXTRACT(YEAR FROM trip_start_timestamp)
* The FROM field can be a little tricky until you are used to it. The format is:
    * A backick (the symbol `).
    * The project name. In this case it is bigquery-public-data.
    * A period.
    * The dataset name. In this case, it is chicago_taxi_trips.
    * A period.
    * The table name. You used this as your answer in 1) Find the data.
    * A backtick (the symbol `).

In [35]:
query = """        
        SELECT EXTRACT(YEAR FROM trip_start_timestamp) as year, count(1) as num_trips
        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`         
        group by year 
        order by num_trips desc
        """
        
df_result = execute_query(client, query)
df_result    

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,year,num_trips
0,2014,37395436
1,2015,32385875
2,2016,31759339
3,2013,27217716
4,2017,24988003
5,2018,20732088
6,2019,16477365
7,2021,3948045
8,2020,3889032
9,2022,2948615


In [38]:
query = """       
        with only_year_table as (
            SELECT EXTRACT(YEAR FROM trip_start_timestamp) as year
            FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`                 
        )
        
        select year, count(1) as num_trips
        from only_year_table
        group by year
        order by num_trips desc
        """
        
df_result = execute_query(client, query)
df_result    

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,year,num_trips
0,2014,37395436
1,2015,32385875
2,2016,31759339
3,2013,27217716
4,2017,24988003
5,2018,20732088
6,2019,16477365
7,2021,3948045
8,2020,3889032
9,2022,2948615


### 4.2 Dive slightly deeper
You'd like to take a closer look at rides from 2017. Copy the query you used above in `rides_per_year_query` into the cell below for `rides_per_month_query`. Then modify it in two ways:


* Use a WHERE clause to limit the query to data from 2017.
* Modify the query to extract the month rather than the year.

In [39]:
query = """       
        with year_month_table as (
            SELECT EXTRACT(YEAR FROM trip_start_timestamp) as year, EXTRACT(MONTH FROM trip_start_timestamp) as month
            FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`                 
        )
        
        select month, count(1) as num_trips
        from year_month_table
        where year = 2017
        group by month
        order by num_trips desc
        """
df_result = execute_query(client, query)
df_result    

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,month,num_trips
0,3,2362105
1,6,2324472
2,5,2323386
3,4,2194702
4,10,2141197
5,8,2079861
6,7,2054299
7,1,1972071
8,9,1950631
9,2,1909802


### 4.3 Write the query

It's time to step up the sophistication of your queries. Write a query that shows, for each hour of the day in the dataset, the corresponding number of trips and average speed.

Your results should have three columns:

* `hour_of_day` - sort by this column, which holds the result of extracting the hour from `trip_start_timestamp`.
* `num_trips` - the count of the total number of trips in each hour of the day (e.g. how many trips were started between 6AM and 7AM, independent of which day it occurred on).
* `avg_mph` - the average speed, measured in miles per hour, for trips that started in that hour of the day. Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. (The value 3600 is used to convert from seconds to hours.)

Restrict your query to data meeting the following criteria:

* a `trip_start_timestamp` between 2017-01-01 and 2017-07-01
* `trip_seconds` > 0 and `trip_miles` > 0
You will use a common table expression (CTE) to select just the relevant rides. Because this dataset is very big, this CTE should select only the columns you'll need to create the final output (though you won't actually create those in the CTE -- instead you'll create those in the later SELECT statement below the CTE).

This is a much harder query than anything you've written so far. Good luck!



In [49]:
query = """       
        with hours_date_table as (
            SELECT EXTRACT(HOUR FROM trip_start_timestamp) as hour_of_day, trip_miles, trip_seconds            
            FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`  
            where EXTRACT(DATE FROM trip_start_timestamp) between '2017-01-01' and  '2017-07-01'
            and trip_seconds>0 and trip_miles>0
        )
        
        select hour_of_day, count(1) as num_trips, 3600*SUM(trip_miles)/SUM(trip_seconds) as avg_mph
        from hours_date_table        
        group by hour_of_day
        order by num_trips desc
        """
df_result = execute_query(client, query)
df_result    

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,hour_of_day,num_trips,avg_mph
0,18,771655,13.644524
1,17,760018,12.468464
2,16,704688,14.284181
3,19,704468,16.629013
4,15,643397,15.680145
5,13,632813,17.649133
6,14,625203,16.958042
7,12,624802,17.819459
8,20,601622,19.514650
9,11,596802,18.915092
